# Aula 2 - Técnicas de Seleção de Atributos - parte 1

Na última aula, falamos sobre métodos de redução de dimensionalidade. Hoje vamos continuar com esse tema, porém com uma abordagem diferente. Ao contrário de reduzirmos a dimensionalidade pela combinação de atributos em novos eixos cartesianos, vamos promover a eliminação definitiva das colunas. Essa técnica é conhecida como *Seleção de Atributos*

Nesta aula falaremos sobre:

- Introdução e Tipos de Técnicas de Seleção de atributos
- *Filtering*
- *Embedding* - modelos lineares
- *Embedding* - models baseados em árvores

# Introdução e Tipos de Seleção de Atributos

De acordo com o artigo [desse site](https://medium.com/@mxcsyounes/hands-on-with-feature-selection-techniques-an-introduction-1d8dc6d86c16) - na verdade é uma série bem interessante de artigos, selecionar atributos se trata de uma atividade crucial na etapa de modelagem, visto que com o advento do BigData, temos cada vez mais acesso a dados altamente dimensionais. No entanto, muitas dessas dimensões podem ser ruidosas ou então inúteis para o propósito de modelagem.

O processo de **seleção de atributos** consiste na escolha, com base em algum critério quantitativo, de um **subconjunto** de atributos, menor que o conjunto original, de forma que o modelo treinado com esse subconjunto proporcione um desempenho comparável ao modelo treinado com todas os atributos.

<img src=https://miro.medium.com/max/694/0*D_jQ5yBsvCZjEYIW width=400>

Apesar de ser possível (e recomendado) aplicar o conhecimento de negócios para selecionar atributos, muitas vezes são necessárias técnicas mais automatizadas para selecionar os atributos. Dentre elas, vamos comentar três tipos.

- Técnicas de filtro
- Técnicas de Envelopamento (*Wrapping*)
- Técnicas "Embutidas" (*Embedding*)

## **1. Técnicas de Filtro**

São as técnicas de seleção de atributos que independem de modelos e cuja tomada de decisão funciona basicamente considerando-se as características dos atributos. Os atributos são filtrados antes do processo de aprendizado se iniciar.

### **1.1. Vantagens**

- podem ser utilizados por qualquer algoritmo / modelo
- não são custosos computacionalmente
- excelentes para detectar e eliminar atributos com problemas básicos de qualidade
    - irrelevantes
    - redundantes
    - constantes
    - duplicações
    - correlacionados

### **1.2. Desvantagens**

- Por serem independentes do algoritmo utilizado, tendem a ignorar o efeito dos atributos sobre o desempenho dos modelos
- Além disso, realiza a seleção de atributos de forma individual, de forma que não consegue avaliar o efeito de combinação de atributos - muitas vezes conhecido como "efeitos de interação".

### **1.3. Subdivisões**

Os métodos de filtro podem ainda ser subdivididos em dois tipos:

- **univariados** - quando levam consideração apenas as estatísticas de uma única variável, como por exemplo, exclusão de colunas por variância nula.
- **multivariados** - quando considera a interação entre a variável preditora e a variável alvo, como por exemplo, excluir variáveis que possuem baixa correlação com a variável alvo.

Os métodos de filtro são técnicas de seleção que podem ser supervisionadas ou não, ou seja, podem depender da presençada variável alvo ou não. Além disso, a técnica de selação vai depender do tipo de variável dependente e independente.

<img src="https://machinelearningmastery.com/wp-content/uploads/2019/11/How-to-Choose-Feature-Selection-Methods-For-Machine-Learning.png" text='machinelearningmastery.com/feature-selection-with-real-and-categorical-data/' width=600px>

Essencialmente, os atributos são ranqueados de acordo com a técnica estatística aplicada e, geralmente, escolhem-se os top $k$ atributos, onde $k$ é um hiperparâmetro modificável.

[Link da Documentação de Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)

Vejamos alguns exemplos de aplicação de métodos de filtros.

In [1]:
import warnings
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2
from sklearn.preprocessing import OrdinalEncoder

# ignorar warnings
warnings.filterwarnings('ignore')

In [2]:
# carregar dados
data_reg = pd.read_csv('./data/garments_worker_productivity.csv')
data_reg.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [3]:
# verificando tipos
data_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

In [4]:
# selecionando apenas features numéricos
data_reg = data_reg.select_dtypes(include=np.number)
data_reg.head()

,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [5]:
# analisando-se a variância
data_reg.describe()

,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
count,1197.000000,1197.000000,1197.000000,691.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000,1197.000000
mean,6.426901,0.729632,15.062172,1190.465991,4567.460317,38.210526,0.730159,0.369256,0.150376,34.609858,0.735091
std,3.463963,0.097891,10.943219,1837.455001,3348.823563,160.182643,12.709757,3.268987,0.427848,22.197687,0.174488
min,1.000000,0.070000,2.900000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.233705
25%,3.000000,0.700000,3.940000,774.500000,1440.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.650307
50%,6.000000,0.750000,15.260000,1039.000000,3960.000000,0.000000,0.000000,0.000000,0.000000,34.000000,0.773333
75%,9.000000,0.800000,24.260000,1252.500000,6960.000000,50.000000,0.000000,0.000000,0.000000,57.000000,0.850253
max,12.000000,0.800000,54.560000,23122.000000,25920.000000,3600.000000,300.000000,45.000000,2.000000,89.000000,1.120437


In [13]:
# extraindo features
x = data_reg.drop(['actual_productivity'], axis=1)

In [14]:
# filtrando-se pelo limite de variância
selector = VarianceThreshold(threshold=0.1)
selector.fit(x)

VarianceThreshold(threshold=0.1)

In [15]:
selector.get_support()

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True])

In [16]:
# selecionando os atributos
x = x.loc[:, list(selector.get_support())]
x.head()

,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,8,26.16,1108.0,7080,98,0.0,0,0,59.0
1,1,3.94,NaN,960,0,0.0,0,0,8.0
2,11,11.41,968.0,3660,50,0.0,0,0,30.5
3,12,11.41,968.0,3660,50,0.0,0,0,30.5
4,6,25.90,1170.0,1920,50,0.0,0,0,56.0


In [19]:
# analisando-se a correlação das features com o target
data = x.copy()
data['target'] = data_reg['actual_productivity']

# correlação
data_corr = pd.DataFrame(abs(data.corr(method='spearman')['target']).values,
                         columns=['r'], index = data.corr(method='spearman').index)

# selecionando as features
selec_features = data_corr.loc[data_corr.r > 0.15, 'r'].index.values
selec_features = [col for col in selec_features if col != 'target']
selec_features

['team', 'wip', 'incentive', 'no_of_style_change']

In [20]:
# selecionando os atributos pela correlação
x = x.loc[:, list(selec_features)]
x.head()

,team,wip,incentive,no_of_style_change
0,8,1108.0,98,0
1,1,NaN,0,0
2,11,968.0,50,0
3,12,968.0,50,0
4,6,1170.0,50,0


### ***Teste do $\chi^2$ - Selecionar Atributos Categóricos***

Para atributos numéricos, podemos utilizar a correlação de Pearson ou Spearman para aplicar algoritmos de seleção de atributos. No entanto, a correlação pode não fazer sentido quando tratamos de variáveis de caráter categórico. Assim sendo, podemos utilizar outros testes estatísticos para determinar se existe relação entre variáveis categóricas.

O teste é baseado na comparação da contagem cruzada de duas variáveis categóricas - conhecido com o tabela de contingências. A estatística de teste é calculada pela soma:

$$\chi^2_{calc} = \sum_{i=1}^n \frac{(O_i - E_i)^2}{E_i}$$

Onde: $O_i$ é a frequência observada e $E_i$ é a frequência esperada para determinada categoria dentro da população amostrada. Se as frequências observadas forem muito diferentes das frequências esperadas, então isso pode ser um indício de que as variáveis categóricas estejam correlacionadas entre si. A decisão de significância estatística é feita com base na comparação da estatística calculada com o valor esperado proveniente de uma distribuição do $\chi^2$, assumindo-se a hipótese nula sendo verdadeira - muito semelhante ao que é realizado para os testes de hipóteses que aprendemos em estatística.

Assim sendo, podemos usar esse teste estatístico para poder selecionar atributos categóricos que possuam relação com uma variável alvo também categórica.

In [21]:
# carregar dados de classificação
data_classif = pd.read_csv('./data/german_credit_data.csv')
data_classif.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [22]:
# selecionando apenas variáveis categóricas
data_classif = data_classif.select_dtypes(exclude=np.number)
data_classif.head()

,Sex,Housing,Saving accounts,Checking account,Purpose,Risk
0,male,own,NaN,little,radio/TV,good
1,female,own,little,moderate,radio/TV,bad
2,male,own,little,NaN,education,good
3,male,free,little,little,furniture/equipment,good
4,male,free,little,little,car,bad


In [23]:
# separando predtitores e preditos
x = data_classif.drop(['Risk'], axis=1)
y = data_classif[['Risk']]

# preenchendo os valores ausentes
x = x.fillna('Unknown')

# codificando variáveis categóricas
cod = OrdinalEncoder().fit(x)

In [24]:
# transformando
x_cod = pd.DataFrame(cod.transform(x), columns=x.columns)
y_cod = y['Risk'].map({'good': 0, 'bad': 1})

In [29]:
# selecionando os mais relacionados
selector = SelectKBest(chi2, k=1).fit(x_cod, y_cod)

# mostrando top 2 correlacionados
x_cod.loc[:, selector.get_support()]

,Checking account
0,1.0
1,2.0
2,0.0
3,1.0
4,1.0
...,...
995,0.0
996,1.0
997,0.0
998,1.0


<hr>

**Exercício 1.** Usando o método `SelectKBest` e o dataset `heart.csv`, realize experimentações com diferentes números de atributos para determinar qual a melhor quantidade de atributos a serem utilizadas para classificar a chance de uma pessoa ter ataque cardíaco. Compare o desempenho de um modelo de Regressão Logística e um modelo de Random Forest. *Dica:* por se tratar de um modelo de classificação, use a função `f_classif` dentro do método SelectKBest.

<hr>

## **2. Técnicas Embutidas** (***Embedding***)

Como o próprio nome diz, os métodos embutidos ou *embedded* são aqueles que a seleção de atributos é incluída no treinamento do modelo de Machine Learning. Isso permite que o próprio algoritmo de aprendizagem selecione seus próprios atributos mais importantes, **durante** o treinamento do modelo, ao contrário de testar os atributos individualmente ou de testar combinações previamente selecionadas dos atributos.

### **2.1. Vantagens**

- podem levar em consideração interações entre as variáveis
- são rápidos como os métodos de filtro, porém são mais acurados
- menor tendência a *overfitting*

### **2.2. Desvantagens**

- o conjunto de atributos depende do modelo aplicado

### **2.3. Técnicas Embutidas para Modelos Lineares - Regularização LASSO**

A regularização LASSO é uma técnica de regularização dos modelos lineares que serve como seletora de atributos, visto que a forma da penalização aplicada à função de custo força os coeficientes dos atributos menos importantes a serem zero. Relembrando a forma da regularização LASSO:

$$LOSS_{LASSO} = MSE + \frac{1}{C} \sum_{i=1}^k |\beta_{k}|$$

Onde: $C$ é o coeficiente que regula a força da regularização; $\beta_k$ são os coeficientes da equação linear

<img src=https://ugc.futurelearn.com/uploads/assets/2b/fe/2bfe399e-503e-4eae-9138-a3d7da738713.png width=800>

Embora ambas as modalidades de regularização tenham sido introduzidas com o intuito de simplificar o espaço de hipóteses, o LASSO faz isso de maneira explícita, efetivamente possibilitando a realização de feature selection!

No entanto, há um problema: são poucos os métodos que têm o LASSO incorporado (ex.: regressão linear, logística, XGBoost).

Assim, se quisermos realizar feature selection utilizando outros estimadores, precisamos de técnicas mais genéricas, que foi o que vimos.

Para utilizarmos o L1, uma abordagem possível é:

- **treinar inicialmente um modelo com LASSO**; 
- identificar quais features **ainda estão presentes no modelo** (isto é, com `coef_` não nulo);
- utilizar apenas estas features para treinar o estimador desejado.

Vejamos um exemplo:

In [73]:
# escalonamento o dataset de regressão
data_reg = pd.read_csv('./data/garments_worker_productivity.csv')
data_reg.head()

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [74]:
# selecionando apenas features numéricos
data_reg = data_reg.select_dtypes(include=np.number)

In [75]:
# extraindo features
x = data_reg.drop(['actual_productivity'], axis=1)

In [76]:
# escalonando atributos
x = pd.DataFrame(StandardScaler().fit_transform(x), columns=x.columns)

In [ ]:
from sklearn.linear_model import Lasso

alphas = []
coefs = []
test_scores = []
train_scores = []

for alpha in np.arange(0, 0.002, 0.00001):
    # Cria uma instância do Lasso Regression
    lasso = Lasso(alpha=alpha)
    
    # Fit Lasso model
    lasso.fit(X_train_sc, y_train)
    
    # Salva os scores do modelo (nesse caso é o coeficiente de determinação R²)
    train_scores.append(lasso.score(X_train_sc, y_train))
    test_scores.append(lasso.score(X_test_sc, y_test))

    # Salva o valor de alpha usado
    alphas.append(alpha)

    # Salva o valor dos coeficientes estimados
    coefs.append(lasso.coef_)

In [ ]:
# Concatena os valores de alpha e dos coeficientes em uma única lista
concat_data = [np.append(alphas[i], coefs[i]) for i in range(len(alphas))]

# Cria dataframe com um valor de lambda por linha e as colunas como valores dos coeficientes
df = pd.DataFrame(concat_data, columns=['lambda']+bc.feature_names.tolist())
df.head(10)

Aqui já podemos ver que alguns coeficientes foram zerados pela regularização. Vejamos o heatmap

Temos muitas variáveis independentes correlacionadas e já sabemos que isso impacta no nosso modelo de regressão linear. Lasso escolhe aleatóriamente uma das variáveis multicolineares e zera as demais. Isso pode impactar na interpretabilidade do nosso modelo.

Vamos ver como fica o coeficiente de determinação tanto para o treino quanto para o teste conforme aumentamos nossa regularização $\lambda$.

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(x=alphas, y=train_scores, label='Train')
sns.lineplot(x=alphas, y=test_scores, label='Teste')
plt.ylabel('Coeficiente de determinação (R²)')
plt.xlabel('Lambda')
plt.title('Qual o impacto da regularização Lasso no erro de predição do treino e do teste?');

Conforme esperado, o R² do treino diminui com o aumento da regularização enquanto o do teste atinge um plato próximo de 0.75 e depois começa a cair. Vamos ver agora como o aumento da regularização afeta os coeficientes das nossa features:

In [ ]:
x_lim = [-0.00001, 0.0012]
figure_size = (15,7)
df.plot(x='lambda', figsize=figure_size)
plt.axvline(x=0.00015, linestyle='--')
plt.xlim(x_lim)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.ylabel('Coeficientes encontrados')
plt.xlabel('Lambda')
plt.title('Qual o impacto da regularização Lasso nos coeficientes?', size=16);

Também podemos ver quantas features sobram conforme aumentamos a regularização:

In [ ]:
qtd_features_zeradas = df.groupby('lambda').agg(lambda x: x.ne(0).sum()).sum(axis=1).reset_index()
plt.figure(figsize=figure_size)
sns.lineplot(data=qtd_features_zeradas, x='lambda', y=0)
plt.xlim(x_lim)
plt.ylim([0, 33])
plt.ylabel('Coeficientes não zerados')
plt.xlabel('Lambda')
plt.title('Qual a quantidade de coeficientes zerados com o aumento da regularização Lasso?', size=16);

### **2.4. Técnicas Embutidas para Árvores - Importância de Atributos**

Além de estimadores poderosos, podemos utilizar modelos baseados em árvores para fazer feature selection! 

Há duas formas comuns de utilizarmos árvores para a determinação da importância de features.

#### `.feature_importances_`

Neste caso, o score de importância de cada uma das features é calculado com base na **média e desvio padrão da diminuição de impureza que cada feature proporciona na árvore (ou em cada árvore, no caso de ensembles)**.

O método é conhecido como **mean decrease in impurity** (MDI).

Este método é rápido, no entanto, o valor é fortemente enviesado para features que têm alta cardinalidade (features numéricas, ou features categóricas com muitos níveis).

Neste caso, é melhor utilizar o método de permutation feature importance. Para uma comparação detalhada entre os dois métodos, [veja esta página](https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# instancia e faz o fit do RF
rf = RandomForestClassifier(n_estimators=50,
                            random_state=42).fit(X_train, y_train)

In [ ]:
# Calcula a média para cada feature
np.mean([tree.feature_importances_ for tree in rf.estimators_], axis=0)

In [ ]:
# Calcula o desvio padrão para cada feature
std_fis_rf = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
std_fis_rf

In [ ]:
# cria series com o nome da feature e o importance
feature_importances_rf = pd.Series(rf.feature_importances_, index=rf.feature_names_in_).sort_values(ascending=False)

feature_importances_rf

In [ ]:
# plota o feature importance
plt.figure(figsize=(12, 7))
plt.title("Feature importances using MDI")
plt.barh(feature_importances_rf.index, feature_importances_rf.values)
plt.xlabel("Mean decrease in impurity");

Vamos dar uma olhada na variação...

In [ ]:
plt.figure(figsize=(12, 7))

plt.title("Feature importances using MDI")

# aqui, mesmo plot, mas com as barras de erro (desvio padrão que calculamos acima)
plt.barh(feature_importances_rf.index, feature_importances_rf.values, xerr=std_fis_rf)

plt.xlabel("Mean decrease in impurity")

plt.show()

A variação é enorme! O que pode ter acontecido?

Isso se deve justamente ao viés indesejado que é introduzido pelo MDI. Para corrigir isso, vamos introduzir um novo método na próxima aula.

<hr>

**Exercício 2.** Aplique o modelo LASSO no dataset `german_credit_data` para selecionar atributos. Utilize a métricas `f1_score` para determinar o valor ideal da força da regularização. Ajuste modelos lineares e depois, um modelo AdaBoost para comparar os resultados.

<hr>

**Exercício 3** Faça a mesma análise do exercício 2, porém utilizando o conceito de importância de atributos dos modelos de árvore. Teste diferentes níveis de importância relativa com um modelo GradientBoosting para comparar o desempenho.